# [Multivariate time series forecasting using LSTM](https://github.com/bnsreenu/python_for_microscopists/blob/master/181_multivariate_timeseries_LSTM_GE.py) 

In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler

## To-Do
- split date and time in feature.
- scale
- create a LSTM dataset
- train, fit , predict.

In [35]:
df = pd.read_csv("../Data/cleanned/zusammen.csv")

In [36]:
df.drop(columns=['date','rohwert'], inplace= True)

In [37]:
df = df[['temp', 'dwpt', 'rhum', 'prcp', 'wert']]

In [53]:
trainDf = df[0:23810]

In [24]:
scaler = StandardScaler()
scaler = scaler.fit(df)
df_scaled = scaler.transform(df)